### Test createInitalSkyWcs on ComCam and LSSTCam, April 15, 2025 with latest weekly

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import lines
from mpl_toolkits import axes_grid1
from matplotlib import collections
from tabulate import tabulate

from lsst.afw import cameraGeom
import lsst.geom
import lsst.geom as geom
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D

from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.base import createInitialSkyWcsFromBoresight, createInitialSkyWcs
from lsst.obs.lsst import LsstComCam,LsstCam
camera = LsstCam.getCamera()
comcamera = LsstComCam.getCamera()

from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
import lsst.summit.utils.butlerUtils as butlerUtils
client = makeEfdClient()

# some astropy code for coordinate Transforms
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import Angle as AAngle
from astropy.coordinates import AltAz
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import EarthLocation
#location = EarthLocation(lat=-30.244639*u.deg, lon=-70.749417*u.deg, height=2663*u.m)
location = EarthLocation.of_site('Rubin:Simonyi')
from astropy.coordinates import angular_separation
from astropy.visualization import imshow_norm, MinMaxInterval,AsinhStretch,LinearStretch,SqrtStretch,ContrastBiasStretch,ZScaleInterval,AsymmetricPercentileInterval,ManualInterval


from os import listdir
import healpy as hp
from astropy.table import Table, vstack

# Code

In [ ]:
def radec_to_azalt(ra, dec, location, obstime):
    #Convert RA and dec to Azimuth (deg) and Altitude (deg)    
    radec_coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, obstime=obstime, location=location, frame='icrs')
    return radec_coord.altaz.az.deg, radec_coord.altaz.alt.deg

In [ ]:
# Function to calculate local sidereal time
def calculate_lst(mjd):
    # Define the Rubin Observatory location on Cerro Pachón
    # see https://rubin-obs.slack.com/archives/C07Q45NQN15/p1676639765212759?thread_ts=1676581080.410319&cid=C07Q45NQN15
    location = EarthLocation(lat=-30.244639, lon=-70.749417, height=2663)  # height in meters

    # Create an astropy Time object for the given MJD
    time = Time(mjd, format='mjd', scale='utc')
    
    # Calculate the local sidereal time
    lst = time.sidereal_time('apparent', longitude=location.lon)
    
    return lst

# calculate Parallactic angle using a snippet from https://astroplan.readthedocs.io/en/latest/_modules/astroplan/observer.html
def calculate_q(lst,ra,dec):

    location = EarthLocation(lat=-30.244639, lon=-70.749417, height=2663)

    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst.radian - ra.radian)
    q = np.arctan2(np.sin(H),
               (np.tan(location.lat.radian) * np.cos(dec.radian) -
                np.sin(dec.radian)*np.cos(H)))*u.rad
    return np.rad2deg(q)

In [ ]:
def visinfo_print(im,exprec):
    # print out some of the visit info 
    visinfo = im.visitInfo
    print('BoresightRotAngle: ',visinfo.getBoresightRotAngle().asDegrees())  # this is ROTPA in the header
    print('BoresightParAngle: ',visinfo.getBoresightParAngle().asDegrees())
    print('Visit Az,Alt: ',visinfo.getBoresightAzAlt())
    RaDec = visinfo.getBoresightRaDec()
    print('Ra,Dec: ',RaDec)

    date = visinfo.getDate()
    print('Date: ',date)
    mjd = date.toAstropy()
    print('MJD: ',mjd)
    obstime = Time(mjd, format='mjd', scale='tai')
    print('Astropy time: ',obstime)

    lst = calculate_lst(mjd)
    ra = RaDec.getLongitude().asDegrees()
    dec = RaDec.getLatitude().asDegrees()
    ra_AA = AAngle(ra, unit=u.deg)
    dec_AA = AAngle(dec,unit=u.deg)
    q = calculate_q(lst,ra_AA,dec_AA)
    print('LST: ',lst)
    print('parallactic angle: ',q)

    print('ROTPA fits header: ',im.metadata['ROTPA'])

    location = EarthLocation(lat=-30.244639, lon=-70.749417, height=2663)  # height in meters
    az,alt = radec_to_azalt(ra, dec, location, obstime)
    print('Calculated Az,Alt: ',az,alt)

    rotData = getEfdData(client=client, topic="lsst.sal.MTRotator.rotation", expRecord=exprec)
    if 'actualPosition' in rotData:
        pos = rotData['actualPosition'].values.mean()
        print(f"Mean rotator position was {pos:.2f} degrees")
    else:
        print('no EFD data')

In [ ]:
def plotCCD(imarr,pixrange=None,interpolation='None',title='Title',filename=None,percent=2.5,lo=None,hi=None,filterrad=4.0):


    if pixrange==None:
        p_lo,p_hi = np.nanpercentile(imarr,[percent,100-percent])
    else:
        p_lo,p_hi = np.nanpercentile(imarr[pixrange[1][0]:pixrange[1][1],pixrange[0][0]:pixrange[0][1]],[percent,100-percent])

    if lo==None:
        lo = p_lo
    if hi==None:
        hi = p_hi
    
    print(lo,hi)

    f,ax = plt.subplots(1,1,figsize=(10.,10.))

    im, norm = imshow_norm(imarr, ax, origin='lower', interpolation=interpolation,filterrad=filterrad,cmap='plasma',
                       interval=ManualInterval(lo,hi),
                       stretch=LinearStretch())
    ax.set_title(title)
    add_colorbar(im)

    if pixrange!=None:
        ax.set_xlim(pixrange[0][0],pixrange[0][1])
        ax.set_ylim(pixrange[1][0],pixrange[1][1])

    return f,ax

def add_colorbar(im, aspect=20, pad_fraction=0.5, **kwargs):
    """Add a vertical color bar to an image plot."""
    divider = axes_grid1.make_axes_locatable(im.axes)
    width = axes_grid1.axes_size.AxesY(im.axes, aspect=1./aspect)
    pad = axes_grid1.axes_size.Fraction(pad_fraction, width)
    current_ax = plt.gca()
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.sca(current_ax)
    return im.axes.figure.colorbar(im, cax=cax, **kwargs)


In [ ]:
def make_lsstcam_WCS(camera,visitInfo,location,obstime,extra_rotation=Angle(-90.,geom.degrees)):
    """
    Parameters
    ----------
    camera : lsst.afw.cameraGeom.Camera 
        Camera object
    visitInfo : lsst.afw.image.VisitInfo 
        visit info from an Image
    location: astropy.coordinates.earth.EarthLocation
        Observatory location
    obstime: astropy.time.core.Time
        Time of Observation
    extra_rotation : lsst.geom.Angle
        Added rotation to add to visitInfo.orientation to align the LSSTCam properly

    Returns
    -------
    cam_wcs : dictionary 
        WCS for each detector, keyed by detector Id
    cam_radec : dictionary 
        [RA,dec] at the center of each detector, keyed by detector Id    
    cam_azalt : dictionary 
        [Az,Alt] at the center of each detector, keyed by detector Id       
    """
    orientation = visitInfo.getBoresightRotAngle()
    boresight = visitInfo.getBoresightRaDec()

    orientation_corrected = orientation+extra_rotation
    orientation_corrwrap = orientation_corrected.wrap()   # guessing that this is needed

    # Get WCS and ra,dec and alt,az for each CCD in camera
    cam_wcs = {}
    cam_radec = {}
    cam_azalt = {}

    for det in camera:
        # get WCS
        cam_wcs[det.getId()] = createInitialSkyWcsFromBoresight(boresight, orientation_corrwrap, det, flipX=False)
        # get central pixel
        x0,y0 = det.getBBox().getCenterX(),det.getBBox().getCenterY()  

        # get ra,dec
        ra1,dec1 = cam_wcs[det.getId()].pixelToSky(x0,y0)
        cam_radec[det.getId()] = [ra1.asDegrees(),dec1.asDegrees()]
        cam_azalt[det.getId()] = radec_to_azalt(ra1.asDegrees(),dec1.asDegrees(),location,obstime)

    return cam_wcs,cam_radec,cam_azalt
    

In [ ]:
def get_stars(detector,ra,dec,cam_wcs,npixedge=0):
    # some constants and useful objects
    ccd_diag = 0.15852  #Guider CCD diagonal radius in Degrees
    path = '/home/s/shuang92/rubin-user/Monster_guide'  ## RSP path
    res = 5
    nside = 2**res
    npix = 12 * nside**2
    bad_guideramps = {193: 'C1', 198: 'C1', 201: 'C0'}
    filters = ['u','g','r','i','z','y']

    # get BBox for this detecctor, removing some number of edge pixels
    ccd_bbox = detector.getBBox()
    ccd_bbox.grow(-Extent2I(npixedge, npixedge))
    
    # query the Monster
    hp_ind = hp.ang2pix(nside, ra, dec, lonlat=True)

    # should only need at most 4 tiles, but for simplicity using 9 Tables
    SW, W, NW, N, NE, E, SE, S = hp.get_all_neighbours(nside, hp_ind)
    
    this_table = Table.read(f'{path}/{hp_ind}.csv')

    E_table = Table.read(f'{path}/{E}.csv')
    W_table = Table.read(f'{path}/{W}.csv')
    S_table = Table.read(f'{path}/{S}.csv')
    N_table = Table.read(f'{path}/{N}.csv')

    SW_table = Table.read(f'{path}/{SW}.csv')
    SE_table = Table.read(f'{path}/{SE}.csv')
    NW_table = Table.read(f'{path}/{NW}.csv')
    NE_table = Table.read(f'{path}/{NE}.csv')

    star_cat = vstack([this_table, E_table, W_table, S_table, N_table, SW_table, SE_table, NW_table, NE_table])

    # find the ra,dec of the CCD center
    x0,y0 = detector.getBBox().getCenterX(),detector.getBBox().getCenterY()  
    ra_ccd,dec_ccd = cam_wcs.pixelToSky(x0,y0)

    star_cat['dangle'] = np.degrees(angular_separation(ra_ccd.asRadians(),dec_ccd.asRadians(),
                            star_cat['coord_ra'],star_cat['coord_dec']))
    insideCCDradius = (star_cat['dangle']<ccd_diag) # inside the Guider CCD radius

    # Selection 1: the Star is isolated and inside the CCD radius
    cat_select1 = star_cat[(insideCCDradius)]

    # using the wcs to locate the stars inside the CCD minus npixedge
    ccdx,ccdy = cam_wcs.skyToPixelArray(cat_select1['coord_ra'],cat_select1['coord_dec'],degrees=False)
    inCCD = ccd_bbox.contains(ccdx,ccdy)

    # fill with CCD pixel x,y
    cat_select1['ccdx'] = ccdx
    cat_select1['ccdy'] = ccdy
    cat_select1['inCCD'] = inCCD
    

    return cat_select1

# Get a sample Image

In [ ]:
# just get a raw LSSTCam image 

dayObs=20250415
seqNum = 66
idet = 29  # pick a CCD near the Guiders
collection_lsstcam = collections = ['LSSTCam/raw/all','LSSTCam/runs/nightlyValidation']
butler_lsstcam = Butler("/repo/embargo_new", collections=collection_lsstcam)
imdataId = {'instrument': 'LSSTCam', 'detector': idet, 'day_obs': dayObs,'seq_num':seqNum}
raw_lsstcam = butler_lsstcam.get('raw', imdataId) 
exprecord_lsstcam = butlerUtils.getExpRecord(butler_lsstcam,instrument='LSSTCam',dayObs=dayObs,seqNum=seqNum)

postISRCCD_lsstcam = butler_lsstcam.get('post_isr_image', imdataId) 
calexp_lsstcam = butler_lsstcam.get('preliminary_visit_image', imdataId) 


In [ ]:
print(raw_lsstcam.visitInfo)

In [ ]:
visinfo_print(raw_lsstcam,exprecord_lsstcam)



In [ ]:
radec = raw_lsstcam.visitInfo.getBoresightRaDec()
ra = radec.getRa().asDegrees()
dec = radec.getDec().asDegrees()
print(ra,dec)

print(radec.getRa().asRadians())
print(radec.getDec().asRadians())

In [ ]:
exprecord_lsstcam

In [ ]:
raw_lsstcam.metadata

In [ ]:
f,ax = plotCCD(postISRCCD_lsstcam.getImage().getArray(),title='%d seqNum=%d Id=%d' % (dayObs,seqNum,idet))

# test initWCS 

In [ ]:
# get WCS for lsstcam image

visinfo_lsstcam = raw_lsstcam.visitInfo

date = visinfo_lsstcam.getDate()
print('Date: ',date)
mjd = date.toAstropy()
print('MJD: ',mjd)
obstime = Time(mjd, format='mjd', scale='tai')

camcorr_wcs,camcorr_radec,camcorr_azalt = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
                                                           extra_rotation=Angle(0.,geom.degrees))

#camcorr_wcs_no90,camcorr_radec_no90,camcorr_azalt_no90 = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
#                                                           extra_rotation=Angle(0.,geom.degrees))
#
#camcorr_wcs_plus90,camcorr_radec_plus90,camcorr_azalt_plus90 = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
#                                                           extra_rotation=Angle(90.,geom.degrees))
#
#camcorr_wcs_plus180,camcorr_radec_plus180,camcorr_azalt_plus180 = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
#                                                          extra_rotation=Angle(180.,geom.degrees))

In [ ]:
detector = camera[idet]
star_cat = get_stars(detector,camcorr_radec[idet][0],camcorr_radec[idet][1],camcorr_wcs[detector.getId()])

In [ ]:
star_cat[['coord_ra','coord_dec','mag_i','gaia_G','ccdx','ccdy','inCCD']]

In [ ]:
f,ax = plotCCD(postISRCCD_lsstcam.getImage().getArray(),title='%d sedNum=%d Id=%d initWCS' % (dayObs,seqNum,idet))

okstars = star_cat[(star_cat['inCCD'])]
ax.scatter(okstars['ccdx'],okstars['ccdy'],s=100.0,facecolors='none', edgecolors='red')


# use the calexp wcs which is after a basic level Astrometric fit

In [ ]:
detector = camera[idet]
star_cat2 = get_stars(detector,camcorr_radec[idet][0],camcorr_radec[idet][1],calexp_lsstcam.getWcs())

In [ ]:
f,ax = plotCCD(postISRCCD_lsstcam.getImage().getArray(),title='%d seqNum=%d Id=%d calexp WCS' % (dayObs,seqNum,idet))

okstars = star_cat2[(star_cat2['inCCD'])]
ax.scatter(okstars['ccdx'],okstars['ccdy'],s=100.0,facecolors='none', edgecolors='red')

# Find deltax,deltay pixel offset between initWCS and the calexp WCS

In [ ]:
calwcs = calexp_lsstcam.getWcs()
initwcs = raw_lsstcam.getWcs()

bbox = detector.getBBox()
center = bbox.getCenter()
cal_radec = calwcs.pixelToSky(center)
init_center = initwcs.skyToPixel(cal_radec)
delta_center = center - init_center
print(delta_center)

In [ ]:
bbox.getCenter()

In [ ]:
cal_trans = calwcs.getTransform()
init_trans = initwcs.getTransform()

In [ ]:
calwcs.getCdMatrix()

# try tweaking the initWCS to match the image

In [ ]:
from lsst.afw.geom import makeModifiedWcs,TransformPoint2ToPoint2


In [ ]:
import astshim

In [ ]:
initwcs = camcorr_wcs[detector.getId()]
print(initwcs)

In [ ]:
# adjust WCS in x,y by eye from above image

deltax = -725.0  # in pixels
deltay = -535.0
corrwcs = initwcs.copyAtShiftedPixelOrigin(Extent2D(deltax,deltay))

In [ ]:
# also apply a rotation - not exactly sure where it is rotating about, but looks like its about a point inside the CCD
theta = np.deg2rad(0.1)
rotmat = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
amat = astshim.MatrixMap(rotmat)
tp2p = TransformPoint2ToPoint2(amat)
corrwcs2 = makeModifiedWcs(tp2p,corrwcs,True)
print(corrwcs2)

In [ ]:
detector = camera[idet]
star_cat3 = get_stars(detector,camcorr_radec[idet][0],camcorr_radec[idet][1],corrwcs2)

In [ ]:
f,ax = plotCCD(postISRCCD_lsstcam.getImage().getArray(),title='%d seqNum=%d Id=%d initWCS tweaked' % (dayObs,seqNum,idet))

okstars = star_cat3[(star_cat3['inCCD'])]
ax.scatter(okstars['ccdx'],okstars['ccdy'],s=100.0,facecolors='none', edgecolors='red')

# Focal Plane Detector orientation plot

In [ ]:
# plot Y(up) vs. X(right) as seen through L1 for the LSSTCam detectors
f,ax = plt.subplots(1,1,figsize=(10,10))
for i,azalt in camcorr_azalt.items():
    _ = ax.plot(-azalt[0],-azalt[1],marker='o',markersize=3.,color='red')
    _ = ax.text(-azalt[0],-azalt[1],camera[i].getId(),size=12.)
#ax.set_aspect('equal')
ax.set_title('LSSTCam Y(up) vs. X(right), as seen through L1, from DM WCS with no extra_rotation')

In [ ]:
# get WCS for lsstcam image
expId = 2025050500523
dayObs=20250505
seqNum = 523
idet = 94  # pick a CCD near the Guiders
collection_lsstcam = collections = ['LSSTCam/raw/all','LSSTCam/runs/nightlyValidation']
butler_lsstcam = Butler("/repo/embargo_new", collections=collection_lsstcam)
imdataId = {'instrument': 'LSSTCam', 'detector': idet, 'day_obs': dayObs,'seq_num':seqNum}
raw_lsstcam = butler_lsstcam.get('raw', imdataId) 




visinfo_lsstcam = raw_lsstcam.visitInfo

date = visinfo_lsstcam.getDate()
print('Date: ',date)
mjd = date.toAstropy()
print('MJD: ',mjd)
obstime = Time(mjd, format='mjd', scale='tai')

camcorr_wcs,camcorr_radec,camcorr_azalt = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
                                                           extra_rotation=Angle(0.,geom.degrees))

#camcorr_wcs_no90,camcorr_radec_no90,camcorr_azalt_no90 = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
#                                                           extra_rotation=Angle(0.,geom.degrees))
#
#camcorr_wcs_plus90,camcorr_radec_plus90,camcorr_azalt_plus90 = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
#                                                           extra_rotation=Angle(90.,geom.degrees))
#
#camcorr_wcs_plus180,camcorr_radec_plus180,camcorr_azalt_plus180 = make_lsstcam_WCS(camera,visinfo_lsstcam,location,obstime,
#                                                          extra_rotation=Angle(180.,geom.degrees))

In [ ]:
camcorr_wcs[94]

In [ ]:
camcorr_wcs[201]